In [2]:
import os 
os.getcwd()

'C:\\Users\\boshi\\For Work'

In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
from datetime import datetime

# Enter the code with os here to change current directory 

class TimeRecorderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Time Recorder")

        self.categories = ["Work", "Study", "Break", "Other", "Capstone", "Project"]

        self.create_widgets()
        self.recorded_data = []

    def create_widgets(self):
        # Category dropdown
        self.category_label = ttk.Label(self.root, text="Select Category:")
        self.category_label.pack(pady=10)
        self.category_var = tk.StringVar()
        self.category_dropdown = ttk.Combobox(self.root, textvariable=self.category_var, values=self.categories)
        self.category_dropdown.pack()

        # Start/Stop buttons
        self.start_button = ttk.Button(self.root, text="Start", command=self.start_timer)
        self.stop_button = ttk.Button(self.root, text="Stop", state="disabled", command=self.stop_timer)
        self.start_button.pack(pady=10)
        self.stop_button.pack()
        
        # Recorded data display
        self.data_label = ttk.Label(self.root, text="Recorded Data:")
        self.data_label.pack(pady=10)
        self.data_text = tk.Text(self.root, height=10, width=30)
        self.data_text.pack()
        
        # Export button
        self.export_button = ttk.Button(self.root, text="Export to Excel", command=self.export_to_excel)
        self.export_button.pack(pady=10)

    def start_timer(self):
        self.start_time = datetime.now()
        self.start_button.config(state="disabled")
        self.stop_button.config(state="enabled")

    def stop_timer(self):
        end_time = datetime.now()
        category = self.category_var.get()
        time_elapsed = end_time - self.start_time
        self.recorded_data.append([category, self.start_time, end_time, time_elapsed])
        self.display_recorded_data()
        self.start_button.config(state="enabled")
        self.stop_button.config(state="disabled")

    def display_recorded_data(self):
        self.data_text.delete(1.0, tk.END)
        for data in self.recorded_data:
            self.data_text.insert(tk.END, f"Category: {data[0]}\nStart Time: {data[1]}\nEnd Time: {data[2]}\nTime Elapsed: {data[3]}\n\n")

    def export_to_excel(self):
        if not self.recorded_data:
            messagebox.showwarning("Warning", "No data to export.")
            return

        df = pd.DataFrame(self.recorded_data, columns=["Category", "Start Time", "End Time", "Time Elapsed"])
        file_path = "time_record.xlsx"
        df.to_excel(file_path, index=False)
        messagebox.showinfo("Export Complete", f"Data exported to {file_path}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TimeRecorderApp(root)
    
    # Save Data into Excel
    
    root.mainloop()


# With sub categories

In [7]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pandas as pd
from datetime import datetime

class TimeRecorderApp:
    def __init__(self, root, config_file):
        self.root = root
        self.root.title("Time Recorder")

        self.config_file = config_file
        self.load_categories()

        self.create_widgets()
        self.recorded_data = []

    def load_categories(self):
        try:
            self.categories_df = pd.read_excel(self.config_file)
            self.categories = self.categories_df["Category"].tolist()
            self.subcategories = {category: self.categories_df[self.categories_df["Category"] == category]["Subcategory"].tolist() for category in self.categories}
        except Exception as e:
            messagebox.showerror("Error", f"Error loading categories from the config file: {str(e)}")
            self.categories = []
            self.subcategories = {}

    def create_widgets(self):
        # Category dropdown
        self.category_label = ttk.Label(self.root, text="Select Category:")
        self.category_label.pack(pady=10)
        self.category_var = tk.StringVar()
        self.category_dropdown = ttk.Combobox(self.root, textvariable=self.category_var, values=self.categories)
        self.category_dropdown.pack()

        # Subcategory dropdown
        self.subcategory_label = ttk.Label(self.root, text="Select Subcategory:")
        self.subcategory_label.pack(pady=10)
        self.subcategory_var = tk.StringVar()
        self.subcategory_dropdown = ttk.Combobox(self.root, textvariable=self.subcategory_var, values=[""])
        self.subcategory_dropdown.pack()

        # Start/Stop buttons
        self.start_button = ttk.Button(self.root, text="Start", command=self.start_timer)
        self.stop_button = ttk.Button(self.root, text="Stop", state="disabled", command=self.stop_timer)
        self.start_button.pack(pady=10)
        self.stop_button.pack()

        # Recorded data display
        self.data_label = ttk.Label(self.root, text="Recorded Data:")
        self.data_label.pack(pady=10)
        self.data_text = tk.Text(self.root, height=10, width=30)
        self.data_text.pack()

        # Export button
        self.export_button = ttk.Button(self.root, text="Export to Excel", command=self.export_to_excel)
        self.export_button.pack(pady=10)

        # Populate subcategories based on category selection
        self.category_dropdown.bind("<<ComboboxSelected>>", self.update_subcategories)

    def update_subcategories(self, event):
        selected_category = self.category_var.get()
        subcategories = self.subcategories.get(selected_category, [])
        self.subcategory_dropdown['values'] = subcategories
        self.subcategory_var.set("")

    def start_timer(self):
        self.start_time = datetime.now()
        self.start_button.config(state="disabled")
        self.stop_button.config(state="enabled")

    def stop_timer(self):
        end_time = datetime.now()
        category = self.category_var.get()
        subcategory = self.subcategory_var.get()
        time_elapsed = end_time - self.start_time
        self.recorded_data.append([category, subcategory, self.start_time, end_time, time_elapsed])
        self.display_recorded_data()
        self.start_button.config(state="enabled")
        self.stop_button.config(state="disabled")

    def display_recorded_data(self):
        self.data_text.delete(1.0, tk.END)
        for data in self.recorded_data:
            self.data_text.insert(tk.END, f"Category: {data[0]}\nSubcategory: {data[1]}\nStart Time: {data[2]}\nEnd Time: {data[3]}\nTime Elapsed: {data[4]}\n\n")

    def export_to_excel(self):
        if not self.recorded_data:
            messagebox.showwarning("Warning", "No data to export.")
            return

        df = pd.DataFrame(self.recorded_data, columns=["Category", "Subcategory", "Start Time", "End Time", "Time Elapsed"])
        file_path = "time_record.xlsx"
        df.to_excel(file_path, index=False)
        messagebox.showinfo("Export Complete", f"Data exported to {file_path}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TimeRecorderApp(root, "categories_config.xlsx")
    root.mainloop()
